  ## Data Wrangling with Python

Import needed packages (Pandas and OS). If you do not have pandas installed (i.e., you're working with base python; if you have anaconda, you'll be fine) install using `pip install pandas` at the command line. Here is a link to [`pandas` documentation](https://pandas.pydata.org/docs/)

In [ ]:
import pandas as pd, os, numpy as np, seaborn as sns

Download data (see link from [R tutorial](tidying-data.html), but here it is again): [race/ethnicity according to the 2010 census](https://www.prisonpolicy.org/data/race_ethnicity_gender_2010.xlsx)

Use Python's `os` module to list contents of data directory to verify that you have the data (and you know where it is)

In [ ]:
os.listdir("data")

Use Pandas [`read_excel`](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html) method to read Excel sheet into Pandas DataFrame

In [ ]:
incarceration_data_2010 = pd.read_excel("data/race_ethnicity_gender_2010.xlsx",
                                      sheet_name = "Total",
                                      skiprows = range(4))
incarceration_data_2010.head()

Select row where `Geography == Arizona` and list column names

In [ ]:
az_data = incarceration_data_2010[incarceration_data_2010['Geography'] == 'Arizona'].copy()
az_data.columns

We want only the columns that start with "Incarceration Rate", plus the `Geography` column

In [ ]:
az_incarceration_rate = az_data[['Geography'] + 
                                [col_name for col_name in az_data.columns if "Incarceration rate" in col_name]].copy()
az_incarceration_rate

Create "tidied" dataframe that contains columns for geography, race/ethnicity, and incarceration rate. For an elegant solution that is similar to the solution in R, use [Pandas.DataFrame.melt](https://pandas.pydata.org/pandas-docs/version/1.0.0/reference/api/pandas.DataFrame.melt.html):

In [ ]:
# update column names
az_incarceration_rate.columns = ["geography", "Population at large"] + [value.replace("Incarceration rate: ", "") 
                                      for value in az_incarceration_rate.columns[2:]]
# use melt to create reshaped dataframe
az_incarc_rate_tidy = az_incarceration_rate.melt(id_vars = ['geography'], # columns that should not change
                           var_name = "race_ethnicity", # name of column that will contain old col/var names
                           value_name = "incarceration_rate") # name of column that will contain values
az_incarc_rate_tidy

Alternatively, you can implement it yourself:

In [ ]:
incarceration_rate = az_incarceration_rate.iloc[0, 1:].copy()
geo = np.full(len(incarceration_rate), az_data.Geography.values.copy()) 
race_ethnicity = ['Population at large'] + [col_name.replace('Incarceration rate: ', '') for col_name in az_incarceration_rate.columns[2:]]
az_incarceration_rate_tidy = pd.DataFrame(np.column_stack((geo, race_ethnicity, incarceration_rate)), columns = ['geography', 'race_ethnicity', 'incarceration_rate'])
az_incarceration_rate_tidy

If we want, we can change type of `incarceration_rate` column data to float! You will actually be able to interact with these values without changing the type because the default type in Python is `object`, but including this to be consistent with the R tutorial and demonstrate how you might do this (if you needed to) in Python:

In [ ]:
az_incarc_rate_tidy.incarceration_rate = az_incarc_rate_tidy.incarceration_rate.astype(float)

Sort values in descending order by `incarceration_rate`

In [ ]:
az_incarc_rate_tidy.sort_values("incarceration_rate", ascending = False, inplace = True)

Use [seaborn](https://seaborn.pydata.org/examples/part_whole_bars.html) to plot data.

In [ ]:
sns.set_color_codes("pastel")
ax = sns.barplot(x="incarceration_rate", y="race_ethnicity", data=az_incarc_rate_tidy, color="b")
ax.set(xlabel = "incarceration rate per 100k ", 
       ylabel = "", 
       title = "Incarceration rates per 100k people in Arizona")

Write tidy data to csv file

In [ ]:
az_incarc_rate_tidy.to_csv('data/az_incarceration_rates_2010.csv', sep = ";", index = False)